# Neural Networks for Classification

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as mplot3d
import os

In [3]:
from keras import optimizers
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation
from keras.models import Sequential
from keras.utils import to_categorical
import keras.backend

### Choosing 25 subjects from 68 in the CMU PIE dataset

In [4]:
PIE_subjects = os.listdir("PIE")
selected_subjects = np.random.choice(PIE_subjects, size = 25, replace = False)
train_photos_path = []
test_photos_path  = []
for subject in selected_subjects:
    subject_photos = os.listdir("PIE"+"/"+subject)
    subject_train  = np.random.choice(subject_photos, size = int(0.7*len(subject_photos)), replace=False)
    subject_test   = np.setdiff1d(subject_photos, subject_train)
    for photo in subject_train :
        train_photos_path.append("PIE"+'/'+subject+'/'+photo)
    for photo in subject_test :
        test_photos_path.append("PIE"+'/'+subject+'/'+photo)

train_photos_array =  np.array([plt.imread(photo) for photo in train_photos_path])
test_photos_array =  np.array([plt.imread(photo) for photo in test_photos_path])
print("The selected subjects are :\n", selected_subjects)

The selected subjects are :
 ['33' '22' '28' '16' '5' '21' '57' '41' '18' '47' '8' '63' '49' '46' '66'
 '24' '14' '38' '34' '62' '10' '54' '30' '67' '39']


In [5]:
X_train = train_photos_array
Y_train = np.array([path.split('/')[-2] for path in train_photos_path])
X_test  = test_photos_array
Y_test  = np.array([path.split('/')[-2] for path in test_photos_path])

### reading my selfies

In [6]:
selfies = os.listdir("selfies")
selfies_path = []
for selfie in selfies :
    selfies_path.append("selfies" + '/' + selfie)
train_selfies_path = np.random.choice(selfies_path, size = 7, replace=False)
test_selfies_path  = np.setdiff1d(selfies_path, train_selfies_path)

#train_selfies_array = np.array([plt.imread(selfie) for selfie in train_selfies_name])
#test_selfies_array = np.array([plt.imread(selfie) for selfie in test_selfies_name])

In [7]:
X_train_selfies = np.array([plt.imread(selfie) for selfie in train_selfies_path])
X_test_selfies  = np.array([plt.imread(selfie) for selfie in test_selfies_path])

Y_train_selfies = np.array([69 for i in range(7)])
Y_test_selfies  = np.array([69 for i in range(3)])

In [8]:
_, img_size1, img_size2 = X_train.shape
num_features = img_size1*img_size2

In [9]:
# Add the selfies to X_Train, X_test and Y_train, Y_test
X_train = np.concatenate((X_train, X_train_selfies), axis=0)
X_test  = np.concatenate((X_test, X_test_selfies), axis=0)
Y_train = np.concatenate((Y_train, Y_train_selfies), axis=0)
Y_test  = np.concatenate((Y_test, Y_test_selfies), axis=0)

In [10]:
X_train = X_train.reshape(-1,img_size1, img_size1, 1)
X_test  = X_test.reshape(-1,img_size1, img_size1, 1)

In [11]:
# The overall data
X = np.concatenate((X_train, X_test), axis=0)
Y = np.concatenate((Y_train, Y_test), axis=0)

# Convolution neural network

In [12]:
# Onehot encoding of the class vector
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)

Y_train_bin    = onehot_encoder.fit_transform(Y_train.reshape(len(Y_train), 1))
Y_test_bin     = onehot_encoder.fit_transform(Y_test.reshape(len(Y_test), 1))

In [13]:
keras.backend.clear_session()
# Initialize model
model = Sequential()
# conv_layer1
model.add(Conv2D(filters=20, kernel_size=5, padding="same", input_shape=(img_size1, img_size2, 1)))
model.add(MaxPool2D(pool_size=2, strides=2))

# conv_layer2
model.add(Conv2D(filters=50, kernel_size=5, padding='same'))
model.add(MaxPool2D(pool_size=2, strides=2))

# FC layer
model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))

# output layer
model.add(Dense(26))
model.add(Activation('softmax')) 
model.compile(optimizer=optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 20)        520       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 50)        25050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 50)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 500)               1600500   
_________________________________________________________________
activation (Activation)      (None, 500)               0

In [19]:
# train the model
model.fit(X_train, Y_train_bin, epochs=60, batch_size=100, validation_data=(X_test, Y_test_bin),
              shuffle=True, verbose=1 )

Epoch 1/60
30/30 [==============================] - 7s 250ms/step - loss: 0.1729 - accuracy: 0.9627 - val_loss: 0.3669 - val_accuracy: 0.8978
Epoch 2/60
30/30 [==============================] - 8s 252ms/step - loss: 0.1607 - accuracy: 0.9651 - val_loss: 0.3566 - val_accuracy: 0.8939
Epoch 3/60
30/30 [==============================] - 7s 239ms/step - loss: 0.1510 - accuracy: 0.9692 - val_loss: 0.3408 - val_accuracy: 0.9039
Epoch 4/60
30/30 [==============================] - 7s 250ms/step - loss: 0.1414 - accuracy: 0.9719 - val_loss: 0.3331 - val_accuracy: 0.9070
Epoch 5/60
30/30 [==============================] - 7s 239ms/step - loss: 0.1336 - accuracy: 0.9756 - val_loss: 0.3272 - val_accuracy: 0.9085
Epoch 6/60
30/30 [==============================] - 7s 241ms/step - loss: 0.1257 - accuracy: 0.9766 - val_loss: 0.3207 - val_accuracy: 0.9108
Epoch 7/60
30/30 [==============================] - 7s 235ms/step - loss: 0.1174 - accuracy: 0.9804 - val_loss: 0.3025 - val_accuracy: 0.9154
Epoch 

30/30 [==============================] - 7s 240ms/step - loss: 0.0143 - accuracy: 1.0000 - val_loss: 0.1465 - val_accuracy: 0.9654
Epoch 59/60
30/30 [==============================] - 7s 249ms/step - loss: 0.0140 - accuracy: 1.0000 - val_loss: 0.1482 - val_accuracy: 0.9646
Epoch 60/60
30/30 [==============================] - 7s 244ms/step - loss: 0.0137 - accuracy: 1.0000 - val_loss: 0.1453 - val_accuracy: 0.9646


In [20]:
training_loss, training_acc = model.evaluate(X_train, Y_train_bin)

93/93 [==============================] - 2s 24ms/step - loss: 0.0129 - accuracy: 1.0000


In [21]:
test_loss, test_acc = model.evaluate(X_test, Y_test_bin)

41/41 [==============================] - 1s 27ms/step - loss: 0.1453 - accuracy: 0.9646
